<a href="https://colab.research.google.com/github/ahmed-boutar/interpreting-rule-based-models/blob/main/interpreting_rule_based_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('Hello World')